# EDA & ETL Part

In [59]:
#import needed packages
import os
import pandas as pd
import zipfile as zp
import sqlite3 as sql3
from dotenv import dotenv_values
from IPython.display import display

In [ ]:
#importing our secrets from environment varibales
config = dotenv_values(".env")
os.environ['KAGGLE_USERNAME'] = config.get('KAGGLE_USERNAME')
os.environ['KAGGLE_KEY'] = config.get('KAGGLE_KEY')

In [60]:
#connecting to kaggle via api
from kaggle.api.kaggle_api_extended import KaggleApi
kgl_api = KaggleApi()
kgl_api.authenticate()

In [61]:
#here we declare where our paths is for (main data folder, kaggle dataset, extraction path)
data_path = '../data' #main data path
sqlite_path = '../db' #main db path
kaggle_dataset = 'majedalhulayel/sakani-projects-saudi-arabia' #kaggle dataset path

sakani-projects-saudi-arabia.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
#lets get the data
kgl_api.dataset_download_files(kaggle_dataset, data_path)

In [62]:
#unzip the dataset file and save to new dir
try:
  if os.path.exists(data_path): #if the data folder do exists enter here
    with zp.ZipFile(data_path+'/sakani-projects-saudi-arabia.zip') as data: #take from original path
      data.extractall(data_path) #uzip into the path if exists
      print(f"Done extracting all files to: {data_path}") #message
      
  else: #if the data folder doesn't exists enter here
    print(f'Creating new data folder: {data_path}\n') #message
    os.mkdir(data_path) #create new folder if not exists
    with zp.ZipFile(data_path+'/sakani-projects-saudi-arabia.zip') as data: #take from original path
      data.extractall(data_path) #unzip into the new path
    print(f"Done extracting all files to: {data_path}") #message
except:
  print("Invalid file")

Done extracting all files to: dataset


In [63]:
#lets play with the dataset
df = pd.read_csv("../data/Sakani Projects.csv")
df.head()

,id,location_lat,location_lon,city_id,city_name_ar,city_name_en,region_id,region_key,region_name_ar,region_name_en,...,resource_id,resource_type,subsidizable,type,under_construction_status,unit_types_0,unit_types_1,unit_types_2,use_register_interest_flag,views_count
0,project_1044,24.560109,46.519438,14,الرياض,AR RIYADH,1,riyadh,الرياض,Riyadh,...,1044,projects,1,marketplaces,under_construction,apartment,NaN,NaN,0,1039
1,project_1033,24.793165,46.633768,14,الرياض,AR RIYADH,1,riyadh,الرياض,Riyadh,...,1033,projects,1,marketplaces,under_construction,apartment,NaN,NaN,0,1837
2,project_681,24.058337,42.836919,3025,عفيف,AFIF,1,riyadh,الرياض,Riyadh,...,681,projects,1,marketplaces,NaN,land,NaN,NaN,0,5995
3,project_378,18.446355,42.098271,3274,محايل,MUHAYIL,7,asir,عسير,'Asir,...,378,projects,1,marketplaces,NaN,land,NaN,NaN,0,372872
4,project_386,24.545443,44.404584,716,الدوادمي,AD DUWADIMI,1,riyadh,الرياض,Riyadh,...,386,projects,1,marketplaces,NaN,land,NaN,NaN,0,50459


In [64]:
#lets see some info about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386 entries, 0 to 385
Data columns (total 51 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   id                                  386 non-null    object 
 1   location_lat                        386 non-null    float64
 2   location_lon                        386 non-null    float64
 3   city_id                             386 non-null    int64  
 4   city_name_ar                        386 non-null    object 
 5   city_name_en                        386 non-null    object 
 6   region_id                           386 non-null    int64  
 7   region_key                          386 non-null    object 
 8   region_name_ar                      386 non-null    object 
 9   region_name_en                      386 non-null    object 
 10  region_order_sequence               386 non-null    int64  
 11  availability                        386 non-n

In [65]:
#lets deepdive into the dataset, to learn more
df.columns

Index(['id', 'location_lat', 'location_lon', 'city_id', 'city_name_ar',
       'city_name_en', 'region_id', 'region_key', 'region_name_ar',
       'region_name_en', 'region_order_sequence', 'availability',
       'available_units_count', 'available_units_for_auctions_count',
       'basement', 'bookable', 'city_order_sequence', 'delegated_by_broker',
       'developer_name', 'driver_room', 'elevator', 'group_unit_id',
       'maid_room', 'max_area', 'max_bathroom', 'max_bed_room', 'max_price',
       'max_street_width', 'max_unit_age', 'min_area', 'min_bathroom',
       'min_bed_room', 'min_price', 'min_street_width', 'min_unit_age', 'pool',
       'project_name', 'project_type', 'promoted', 'publish', 'publish_date',
       'resource_id', 'resource_type', 'subsidizable', 'type',
       'under_construction_status', 'unit_types_0', 'unit_types_1',
       'unit_types_2', 'use_register_interest_flag', 'views_count'],
      dtype='object')

In [66]:
#unique unite types
df['unit_types_0'].unique()

array(['apartment', 'land', 'townhouse', 'villa'], dtype=object)

In [67]:
#unique project type
df['project_type'].unique()

array(['offplan_private_land', 'lands_moh_land', 'offplan_moh_land',
       'readymade_private_land', 'readymade_moh_land',
       'lands_private_land', 'advanced_self_construction'], dtype=object)

In [68]:
#unique developer name
df['developer_name'].unique()

array(['شركة المسكن المحدودة',
       'شركة جود العمارة للتطوير والاستثمار العقاري', nan,
       'الشركة الوطنية للإسكان', 'علي شار', 'الرائم للمقاولات',
       'دار واعمار للاستثمار والتطوير العقاري',
       'شركة الأثير للتطوير والاستثمار العقاري',
       'شركة الفاف للتطوير العقاري', 'عبدالرحمن الراشد وأولاده', 'رتال',
       'شركة الأولى لتطوير العقارات', 'ثبات المسكن العقارية',
       'مكيون مطورون عمرانيون', 'شركة سمو العقارية مساهمة مدرجة',
       'شركة درة الذهبية للتطوير العقاري',
       'شركة مرسى البحار للمقاولات العامة', 'شركة أفاق الذهبية المحدودة',
       'شركة عراب التمليك للتطوير العقاري',
       'شركة ساطعة التعمير للتطوير والاستثمار العقاري',
       'شركة تمكين للاستثمار و التطوير العقارى',
       'فرع شركة خطى الخير للتشغيل والصيانة',
       'شركة الحناكي للتطوير العقاري',
       'شركة فيصل عبدالله بن سعيدان للاستثمار والتطويرالعقاري',
       'شركة مساكن أفياء للتطوير والاستثمار العقاري',
       'شركة محمد عبدالعزيز الحبيب وشركاؤه للاستثمار العقاري',
       'شركة دور

In [69]:
#unique region key
df['region_key'].unique()

array(['riyadh', 'asir', 'qassim', 'northern_borders', 'makkah', 'jizan',
       'hail', 'madinah', 'tabuk', 'jawf', 'eastern_province', 'najran',
       'bahah'], dtype=object)

In [70]:
#columns before drop
df.columns

Index(['id', 'location_lat', 'location_lon', 'city_id', 'city_name_ar',
       'city_name_en', 'region_id', 'region_key', 'region_name_ar',
       'region_name_en', 'region_order_sequence', 'availability',
       'available_units_count', 'available_units_for_auctions_count',
       'basement', 'bookable', 'city_order_sequence', 'delegated_by_broker',
       'developer_name', 'driver_room', 'elevator', 'group_unit_id',
       'maid_room', 'max_area', 'max_bathroom', 'max_bed_room', 'max_price',
       'max_street_width', 'max_unit_age', 'min_area', 'min_bathroom',
       'min_bed_room', 'min_price', 'min_street_width', 'min_unit_age', 'pool',
       'project_name', 'project_type', 'promoted', 'publish', 'publish_date',
       'resource_id', 'resource_type', 'subsidizable', 'type',
       'under_construction_status', 'unit_types_0', 'unit_types_1',
       'unit_types_2', 'use_register_interest_flag', 'views_count'],
      dtype='object')

In [ ]:
#rename some columns to more clean naming
df.rename(columns = {'under_construction_status':'construction_status','unit_types_0':'unit_type',\
                     'available_units_for_auctions_count':'available_auctions_units','available_units_count':'available_units'}, inplace=True)

In [ ]:
#clean row-level data
df['developer_name'].fillna('لا يوجد مدخل', inplace=True)
df['publish_date'].ffill(inplace=True) #filling nan values with prev value
df['construction_status'].fillna('no entry', inplace=True)
df['location'] = df['location_lat'].astype(str) +','+ df['location_lon'].astype(str) #create new column to handle the lat,lot location

In [71]:
#un_wanted columns to delete
df.drop(['city_id','region_id','region_key','region_order_sequence','city_order_sequence','group_unit_id','promoted','unit_types_1', \
          'unit_types_2','type','resource_id','resource_type','subsidizable','max_street_width','max_unit_age','max_bathroom','driver_room', \
          'elevator','basement','delegated_by_broker','maid_room','min_bathroom','min_street_width','min_unit_age','pool','publish','use_register_interest_flag', \
          'location_lat', 'location_lon'], axis=1, inplace=True)

In [72]:
#columns after drop
df.columns

Index(['id', 'location_lat', 'location_lon', 'city_name_ar', 'city_name_en',
       'region_name_ar', 'region_name_en', 'availability',
       'available_units_count', 'available_units_for_auctions_count',
       'bookable', 'developer_name', 'max_area', 'max_bed_room', 'max_price',
       'min_area', 'min_bed_room', 'min_price', 'project_name', 'project_type',
       'publish_date', 'under_construction_status', 'unit_types_0',
       'views_count'],
      dtype='object')

In [ ]:
#here we see the dataset shape after cleaning
print(f"Dataset rows/columns: {df.shape}")

In [73]:
#len of the new columns
print(f'Length of Data (After Column Filtered): {len(df.columns)}')

Length of Data (After Column Filtered): 24


In [77]:
#lets save the new data to another file
print(f"Saving the new data to another file: {data_path}/cleaned_data.csv")
df.to_csv(data_path+'/cleaned_data.csv', index=False)

In [ ]:
#lets load our cleand data into sqlite table
try:
  if os.path.exists(sqlite_path): #if the data folder do exists enter here
    engine = sql3.connect(config.get('SQLITE_DB'))
    df.to_sql(config.get('SQLITE_TABLE'), engine, index=False)
    print(f"Loading data into sqlite3 database ...!")

  else: #if the data folder doesn't exists enter here
    print(f'Creating new db folder: {sqlite_path}\n') #message
    os.mkdir(sqlite_path) #create new folder if not exists
    engine = sql3.connect(config.get('SQLITE_DB'))
    df.to_sql(config.get('SQLITE_TABLE'), engine, index=False)
    print(f"Loading data into sqlite3 database ...!")
except Exception as e:
  print(f"Invalid db ... \n{e}")

# Analysis Part

In [78]:
#lets see min/max units available
print(f"Minimum unit available: {df['available_units_count'].min()} units\
        \nMaximum unit available: {df['available_units_count'].max()} units\
        \nRegion with minimum projects: {df['region_name_en'].min()} city\
        \nRegion with maximum projects: {df['region_name_en'].max()} city\
        \nDeveloper with minimum projects: {df['developer_name'].min()} projects\
        \nDeveloper with maximum projects: {df['developer_name'].max()} projects")

Minimum unit available: 0 units        
Maximum unit available: 875 units        
Region with minimum projects: 'Asir city        
Region with maximum projects: Tabuk city        
Developer with minimum projects: أبراج ذهب للتطوير العقاري projects        
Developer with maximum projects: وزارة الإسكان projects


In [104]:
#lets see most developers with specific project requirements
spec_req = df.loc[(df['available_units_count'] > 50) & (df['available_units_count'] <=1000) & (df['min_area'] > 250) & (df['max_bed_room'] <= 2)]

In [105]:
#lets see the result
spec_req

,id,location_lat,location_lon,city_name_ar,city_name_en,region_name_ar,region_name_en,availability,available_units_count,available_units_for_auctions_count,...,max_price,min_area,min_bed_room,min_price,project_name,project_type,publish_date,under_construction_status,unit_types_0,views_count
2,project_681,24.058337,42.836919,عفيف,AFIF,الرياض,Riyadh,1,61,0,...,0.0,556.500000,1,0.0,مخطط 879 - العيدانية,lands_moh_land,2022-06-30,no entry,land,5995
13,project_728,24.543460,43.958677,الدوادمي,AD DUWADIMI,الرياض,Riyadh,1,353,0,...,0.0,580.871898,1,0.0,144 - وادي الرشاء,lands_moh_land,2022-06-30,no entry,land,5253
35,project_729,24.545152,43.902027,الدوادمي,AD DUWADIMI,الرياض,Riyadh,1,65,0,...,0.0,595.500000,1,0.0,مخطط أ/ 228,lands_moh_land,2022-06-30,no entry,land,4290
46,project_974,27.549589,47.726099,قرية (قرية العليا),QARYAT AL 'ULYA,المنطقة الشرقية,Eastern Province,1,86,0,...,0.0,424.050000,1,0.0,مخطط 119/20 - قرية العليا,lands_moh_land,2022-06-30,no entry,land,8083
57,project_980,26.866979,47.119182,قرية (قرية العليا),QARYAT AL 'ULYA,المنطقة الشرقية,Eastern Province,1,173,0,...,0.0,600.000000,1,0.0,مخطط البويبيات - 47/18,lands_moh_land,2022-06-30,no entry,land,2306
68,project_983,24.606367,43.601602,الدوادمي,AD DUWADIMI,الرياض,Riyadh,1,54,0,...,0.0,586.538542,1,0.0,مخطط الصالحية - 130,lands_moh_land,2022-06-30,no entry,land,2579
73,project_695,22.449905,39.097564,مدينة الملك عبدالله الاقتصادية,King Abdullah Economic City,مكة المكرمة,Makkah,1,343,0,...,1599680.0,427.400000,1,494908.0,مدينة الملك عبدالله الاقتصادية - المروج,lands_private_land,2021-10-28,no entry,land,35880
76,project_698,0.000000,22.442068,مدينة الملك عبدالله الاقتصادية,King Abdullah Economic City,مكة المكرمة,Makkah,1,230,0,...,621428.0,280.000000,1,265650.0,مدينة الملك عبدالله الاقتصادية - التالة جاردنز,lands_private_land,2021-10-28,no entry,land,35390
79,project_986,23.476689,44.373787,القويعية,QUWAY'IYAH,الرياض,Riyadh,1,127,0,...,0.0,340.000000,1,0.0,مخطط 139 القويعية,lands_moh_land,2022-06-30,no entry,land,6091
101,project_981,27.101857,47.015506,قرية (قرية العليا),QARYAT AL 'ULYA,المنطقة الشرقية,Eastern Province,1,183,0,...,0.0,575.490000,1,0.0,مخطط ش.ع 172 الرفاع,lands_moh_land,2022-06-30,no entry,land,1751
